In [1]:
import torch
from transformers import BertTokenizer,BertForSequenceClassification,AdamW,get_linear_schedule_with_warmup
import tensorflow as tf
import os
import wget
import pandas as pd
import numpy as np

/home/sauban/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sauban/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sauban/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sauban/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
#Get GPU name
gpu=tf.test.gpu_device_name()
print(gpu)

In [3]:
# If GPU available
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device=torch.device("cuda")
    print('There are %d GPU(s) available.' %torch.cuda.device_count())
    print('We will use the GPU:',torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1650


In [4]:
#torch.cuda.empty_cache()

In [5]:
#Get training and validation data
train=pd.read_csv('./contradictory-my-dear-watson/train.csv')
print(train.shape)
print(train.head())
test=pd.read_csv('./contradictory-my-dear-watson/test.csv')

(12120, 6)
           id                                            premise  \
0  5130fd2cb5  and these comments were considered in formulat...   
1  5b72532a0b  These are issues that we wrestle with in pract...   
2  3931fbe82a  Des petites choses comme celles-là font une di...   
3  5622f0c60b  you know they can't really defend themselves l...   
4  86aaa48b45  ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...   

                                          hypothesis lang_abv language  label  
0  The rules developed in the interim were put to...       en  English      0  
1  Practice groups are not permitted to work on t...       en  English      2  
2              J'essayais d'accomplir quelque chose.       fr   French      0  
3  They can't defend themselves because of their ...       en  English      0  
4    เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร       th     Thai      1  


In [6]:
train.groupby('language').count()

,id,premise,hypothesis,lang_abv,label
language,,,,,
Arabic,401,401,401,401,401
Bulgarian,342,342,342,342,342
Chinese,411,411,411,411,411
English,6870,6870,6870,6870,6870
French,390,390,390,390,390
German,351,351,351,351,351
Greek,372,372,372,372,372
Hindi,374,374,374,374,374
Russian,376,376,376,376,376


In [7]:
#Initialize BertTokenzier
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [8]:
train.iloc[:,1:3].shape

(12120, 2)

In [9]:
#Data Preprocessing and tensor generation
seed=2

#train=train.sample(n=10000,random_state=seed)
tokenized_train=tokenizer.batch_encode_plus(train.iloc[:-1000,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')
labels_train=torch.tensor(train.label.values[:-1000])

#val=val.sample(n=2000,random_state=seed)
tokenized_val=tokenizer.batch_encode_plus(train.iloc[-1000:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')
labels_val=torch.tensor(train.label.values[-1000:])

In [10]:
print(tokenized_train['input_ids'].shape)

torch.Size([11120, 128])


In [11]:
print(tokenized_train.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [12]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

batch_size=4

train_data=TensorDataset(tokenized_train['input_ids'],tokenized_train['attention_mask'],tokenized_train['token_type_ids'],labels_train)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data=TensorDataset(tokenized_val['input_ids'],tokenized_val['attention_mask'],tokenized_val['token_type_ids'],labels_val)
val_sampler=RandomSampler(val_data)
val_dataloader=DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [13]:
#Bert Model transformer with a single sequence classification layer on top
model=BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=3,output_attentions=False,output_hidden_states=False)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [14]:
#Set learning rate
optimizer=AdamW(model.parameters(),lr=2e-5)

In [15]:
epochs=4

#Training steps is no_of_batches*no_of_epochs
total_steps=len(train_dataloader)*epochs

#Learning rate scheduler
scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps = total_steps)

In [16]:
# Function to calculate the accuracy of predictions
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [17]:
import random

seed_val = 12

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):
    
    #Put model into training mode
    model.train()

    total_loss=0

    for step, batch in enumerate(train_dataloader):

        #Unpack the training batch
        b_input_ids = batch[0].to(device)
        b_attention_mask=batch[1].to(device)
        b_token_type = batch[2].to(device)
        b_labels = batch[3].to(device)

        #Clear previously calculated gradients before performing a backward pass
        #model.zero_grad()        //Not sure if useful or not

        #Perform a forward pass and get the loss
        outputs=model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask,labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()

        #Perform backward pass to calculate gradients
        loss.backward()

        #Clip the norm of the gradients to 1.0.
        #Used to prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #Update weights
        optimizer.step()

        #Update learning rate
        scheduler.step()

    #Avg loss over training data for an epoch
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    #### Validation

    #Evaluation mode
    model.eval()

    #Tracking variables 
    val_accuracy=0
    nb_val_steps=0

    #Evaluate data for one epoch
    for batch in val_dataloader:
        
        #Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        #Unpack the inputs from dataloader
        b_input_ids,b_attention_mask, b_token_type, b_labels = batch
        
        #Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():        
            outputs = model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask)
        
        #Get the "logits" output by the model. The "logits" are output values prior to applying an activation function like the softmax.
        logits = outputs[0]
        
        #Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        #Calculate the accuracy for this batch of test sentences.
        tmp_val_accuracy = flat_accuracy(logits, label_ids)
        
        #Accumulate the total accuracy.
        val_accuracy += tmp_val_accuracy

        #Track the number of batches
        nb_val_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(val_accuracy/nb_val_steps))


RuntimeError: CUDA out of memory. Tried to allocate 352.00 MiB (GPU 0; 3.82 GiB total capacity; 2.02 GiB already allocated; 208.50 MiB free; 2.13 GiB reserved in total by PyTorch)